[View in Colaboratory](https://colab.research.google.com/github/opooladz/ece_210_final_project/blob/master/210_final_project_convnet.ipynb)

In [0]:
# Import Libs 
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

In [0]:
# use these func defs. didnt have time to make each cell a class. someone can go ahead and do that if they please  
# Comms Stuff Will add more later
def qammod(m,y):
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  # x = np.array([a,1.*b]).T
  return [x , y] 

def qammod2(m,size):
  y = np.random.randint(m,size=size)
  if(np.all(y>=m)):
    print('qammod: all elements of X must be in range [0,M-1]')
  
  if (~np.all(y == np.fix(y) ) ):
    print("qammod: all elements of X must be integers");
  
  c = np.sqrt(m);
  if( ~(np.all(c == np.fix(c)) and np.all(np.math.log(c, 2) == np.fix(np.math.log(c, 2))) ) ):
    print("qammod: M must be a square of a power of 2");
  
  b = -2*np.fmod(y,c) + c - 1.;
  a = 2*np.floor(y *1./c) - c + 1.;
  x = a + 1.*1j*b
  # lets hold x in a np array([real,imag])
  #x = np.array([a,1.*b]).T
  return [x , y]                        

def channel2(m,sequenceLen,trials,Noise,SNRdB):
  [x,y] = qammod2(m,sequenceLen)
  h = [1, 0.5]
  #h = [1]
  xh = np.convolve(x.flatten(),h)   
  SNR = 10**(SNRdB/10)
  sym_noise_pow = np.var(xh)/SNR
  sym_noise_scale_fact = np.sqrt(sym_noise_pow/2)
  if(Noise):
    AWGN = sym_noise_scale_fact*np.random.normal(1, size=(xh.shape[0],trials)) 
    xhN = xh[:,np.newaxis] + AWGN
  #return [xhN.T , np.repeat(y,2) ]
  return [xhN.T ,y ]
  
                        
data={}
def generateData2(m,sequenceLen,trials,Noise,SNRdB):
  [data['x_train'],data['y_train']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB)
  [data['x_val'],data['y_val']] = channel2(m=m,sequenceLen=int(sequenceLen**0.5),trials=trials,Noise= Noise,SNRdB=SNRdB)
  [data['x_test'],data['y_test']] = channel2(m=m,sequenceLen=sequenceLen,trials=trials,Noise=Noise,SNRdB=SNRdB)

In [0]:
# Visulizing the data

m = 16         # order of modulation
seqLen = 64    # length of one trial must be square powers of 2 to make sense see below 
trials = 1000   # number of trials 
Noise = 1
SNRdB = 25 


x = 4
square_power_of_two = 2**(2*x)
seqLen = square_power_of_two  

generateData2(m,seqLen,trials,Noise,SNRdB)
print(data['x_train'].shape) # data  # axis 1 should be 2x seqLen because of conv with a 2tap filter
print(data['y_train'].shape) # labels
print(data['x_train'].reshape(-1,2)) # so u can see the data 

tmp = (data['x_train'][:,:-1].reshape(trials,seqLen))

#plot 1
plt.scatter(np.real(tmp[0][:]),np.imag(tmp[0][:]))   
                                         # I only plot the first 10 points from 
                                         # the sequence because as time goes on 
                                         # the data becomes less dependent on the
                                         # previous symbols. Even 10 is a lot. 
plt.title('16-QAM Mod With Multi-Channel AWGN plot 1')
plt.show()

#plot 2
# Just a pure 16 qam with 1 point on each constalation point 
# clearly we have 16 classes
# [xtmp,ytmp]=  qammod(16,data['y_train'])
# plt.scatter(xtmp[:,0],xtmp[:,1])
# plt.title('16-QAM Mod plot 2')
# plt.show()



In [0]:
# Some keras stuffs
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import Reshape, Permute
from keras import regularizers
import keras
ykeys = {'y_train','y_val','y_test'}
xkeys = {'x_train','x_val','x_test'}

datatmp={}
for i in ykeys:
  datatmp[i] = data[i].flatten()
for i in xkeys:
  datatmp[i] = data[i].reshape(-1,2)
  
y_train = keras.utils.to_categorical(datatmp['y_train'] , num_classes=16)
y_val = keras.utils.to_categorical(datatmp['y_val'] , num_classes=16)
y_test = keras.utils.to_categorical(datatmp['y_test'] , num_classes=16)

print(data['x_train'].shape)
print(data['x_test'].shape)
print(y_train.shape)
print(data['x_val'].shape)


default_data = {
  'x_train': data['x_train'],
  'x_val': data['x_val'],
  'x_test': data['x_test'],
  'y_train': y_train,
  'y_val': y_val,
  'y_test': y_test,
}
